In [1]:
import pandas as pd
import numpy as np
#import geopandas as gpd
import seaborn as sns 
#import geopandas as gpd
#import folium
import warnings
warnings.filterwarnings("ignore")
import re
import matplotlib.pyplot as plt
from sklearn import preprocessing
from collections import Counter
from statsmodels.formula.api import *

In [2]:
data_all = pd.read_csv("iowa_Liquor_Sales.csv")
print("The entire dataset dimension: ",data_all.shape) #19666763, 24)
print("Attributes:",data_all.columns) # 19666763, 24)

The entire dataset dimension:  (19666763, 24)
Attributes: Index(['Invoice/Item Number', 'Date', 'Store Number', 'Store Name', 'Address',
       'City', 'Zip Code', 'Store Location', 'County Number', 'County',
       'Category', 'Category Name', 'Vendor Number', 'Vendor Name',
       'Item Number', 'Item Description', 'Pack', 'Bottle Volume (ml)',
       'State Bottle Cost', 'State Bottle Retail', 'Bottles Sold',
       'Sale (Dollars)', 'Volume Sold (Liters)', 'Volume Sold (Gallons)'],
      dtype='object')


In [3]:
# check missing values 
print("Missing values:\n",data_all.isnull().sum()) 

Missing values:
 Invoice/Item Number            0
Date                           0
Store Number                   0
Store Name                     0
Address                    79927
City                       79926
Zip Code                   79971
Store Location           1886023
County Number             156731
County                    156729
Category                   16974
Category Name              25040
Vendor Number                  5
Vendor Name                    3
Item Number                    0
Item Description               0
Pack                           0
Bottle Volume (ml)             0
State Bottle Cost             10
State Bottle Retail           10
Bottles Sold                   0
Sale (Dollars)                10
Volume Sold (Liters)           0
Volume Sold (Gallons)          0
dtype: int64


In [4]:
missing_records_num = data_all[data_all.isnull().any(axis=1)].shape[0]
missing_records_num #95103
print(f"Missing value percentage is {100*missing_records_num / data_all.shape[0]}%.") #10.062037153750213
print(f"Total missing values is {sum(data_all.isnull().sum())}.") #2481359
print(f"Average number of null values per record" \
      f" is {sum(data_all.isnull().sum()) / missing_records_num}.") #1.253922805712533.

Missing value percentage is 10.062037153750213%.
Total missing values is 2481359.
Average number of null values per record is 1.253922805712533.


In [5]:
# drop records with missing values 
data_all = data_all.dropna()

In [6]:
#top_three_category = l.iloc[:3, 0].to_list()
#data = data_all.loc[data_all["Category"].isin(top_three_category),:]
data = data_all
data.Category.value_counts()
print(data.shape)#(17687886, 24)

(17687886, 24)


# Pre-processing

In [7]:
county_counts = data.groupby("County").size().shape[0] #county counts 200
real_county_counts = data.groupby("County Number").size().shape[0] #99

data["County"] = data["County"].str.title() #case sensitive
exist_county_counts = data.groupby("County").size().shape #103

data["City"] = data["City"].str.title() #case sensitive

In [8]:
#pd.set_option('display.max_rows', 10)
county_countN_df = data.loc[:,["County","County Number","Date"]].groupby(["County","County Number"]).count().reset_index()
print("Counties with the same county number but conflicting names")
county_countN_df["County Number"].value_counts()
print(county_countN_df.loc[county_countN_df["County Number"].isin([71.0, 78.0,17.0, 11.0]),:])

data.County = data.County.replace(["Buena Vist","Cerro Gord",
                                   "O'Brien","O'brien","Pottawatta"], 
                                  ["Buena Vista","Cerro Gordo",
                                    "Obrien","Obrien", "Pottawattamie"])

Counties with the same county number but conflicting names
           County  County Number    Date
10     Buena Vist           11.0   71360
11    Buena Vista           11.0   89480
17     Cerro Gord           17.0  211992
18    Cerro Gordo           17.0  228400
72        O'Brien           71.0   63180
73         Obrien           71.0   69812
80     Pottawatta           78.0  296991
81  Pottawattamie           78.0  299164


In [9]:
# convert data types 
data["Date"]  = pd.to_datetime(data['Date'],format='%m/%d/%Y')
data['year'] = pd.DatetimeIndex(data['Date']).year
data['month'] = pd.DatetimeIndex(data['Date']).month
data["month"] = data["month"].astype("category")
data['day'] = pd.DatetimeIndex(data['Date']).day
data["day"] = data["day"].astype("category")
data["week_of_day"] = data["Date"].dt.dayofweek+1
data["week_of_day"] = data["week_of_day"].astype("category")

data["Store Number"] = data["Store Number"].astype("category")
data["Item Number"] = data["Item Number"].astype("category")
data["County Number"] = data["County Number"].astype("int")
data["County Number"] = data["County Number"].astype("category")
data["Category"] = data["Category"].astype("int")
data["Category"] = data["Category"].astype("category")
data["City"] = data["City"].astype("category")
data["Sale (Dollars)"] = data["Sale (Dollars)"].astype("str")
data["State Bottle Cost"] = data["State Bottle Cost"].astype("str")
data["State Bottle Retail"] = data["State Bottle Retail"].astype("str")

data.loc[:,"Sale (Dollars)"] = data.loc[:, "Sale (Dollars)"].str[1:]
data.loc[:,"State Bottle Cost"] = data.loc[:, "State Bottle Cost"].str[1:]
data.loc[:,"State Bottle Retail"] = data.loc[:, "State Bottle Retail"].str[1:]
data["Sale (Dollars)"] = data["Sale (Dollars)"].astype("float")
data["State Bottle Cost"] = data["State Bottle Cost"].astype("float")
data["State Bottle Retail"] = data["State Bottle Retail"].astype("float")

In [10]:
# covert string representation of city to interger representation
encoder = preprocessing.LabelEncoder().fit(data.City)
data["encoded_city"] = encoder.transform(data.City)
data["encoded_city"] = data["encoded_city"].astype("category")

In [11]:
data[data["year"].isin([2019, 2020])].year.value_counts()

2019    2158261
2020    1760082
Name: year, dtype: int64

In [12]:
# remove potential errors 
cond1 = data["Bottle Volume (ml)"] == 0
cond2 = data["State Bottle Cost"] == 0
cond3 = data["State Bottle Retail"] == 0
cond4 = data["Sale (Dollars)"] == 0
cond5 = data["Bottles Sold"] ==0
cond6 = data["Volume Sold (Liters)"] ==0
data = data.loc[~(cond1|cond2|cond3|cond4|cond5|cond6),:] 
data = data[data["year"].isin([2019, 2020])]
data.shape
# (3448996, 29)

(3448996, 29)

In [13]:
data.year.value_counts()

2019    1928348
2020    1520648
Name: year, dtype: int64

In [14]:
data["Store Loc"] = data["Store Location"].map(lambda x:str(re.findall(r"\((.+?)\)",x)[0]).split(" "))
data["latitude"] = data["Store Loc"].map(lambda x:x[0])
data["longitude"] = data["Store Loc"].map(lambda x:x[1])
data

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Volume Sold (Liters),Volume Sold (Gallons),year,month,day,week_of_day,encoded_city,Store Loc,latitude,longitude
8487396,INV-17914200177,2019-03-05,2590,Hy-Vee Food Store #5 / Cedar Rapids,3235 Oakland Road NE,Cedar Rapids,52402,POINT (-91.658105 42.010971),57,Linn,...,1.50,0.39,2019,3,5,2,61,"[-91.658105, 42.010971]",-91.658105,42.010971
8539434,INV-17962600006,2019-03-06,2536,HY-VEE / PLEASANT HILL,4815 Maple Drive,Pleasant Hill,50317,POINT (-93.519187 41.598109),77,Polk,...,10.50,2.77,2019,3,6,3,323,"[-93.519187, 41.598109]",-93.519187,41.598109
8548730,INV-18429300129,2019-03-28,2648,Hy-Vee #4 / WDM,555 S 51st St,West Des Moines,50265,POINT (-93.773557 41.561197),77,Polk,...,2.25,0.59,2019,3,28,4,429,"[-93.773557, 41.561197]",-93.773557,41.561197
8549200,INV-18057100023,2019-03-11,3712,Monte Spirits,109 N 4th St,Montezuma,50171,POINT (-92.523692 41.585429),79,Poweshiek,...,3.00,0.79,2019,3,11,1,272,"[-92.523692, 41.585429]",-92.523692,41.585429
8555249,INV-17931700030,2019-03-05,4902,Broadway Liquor,821 Broadway St,Waterloo,50703,POINT (-92.345144 42.513772),7,Black Hawk,...,1.50,0.39,2019,3,5,2,418,"[-92.345144, 42.513772]",-92.345144,42.513772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18767245,INV-17817900010,2019-02-27,5371,State Food Mart,3715 State St.,Bettendorf,52722,POINT (-90.482018 41.528972),82,Scott,...,9.00,2.37,2019,2,27,3,40,"[-90.482018, 41.528972]",-90.482018,41.528972
18768153,INV-17822500011,2019-02-27,4568,Select Mart / Sioux City,4103 Floyd Blvd,Sioux City,51108,POINT (-96.358413 42.54202),97,Woodbury,...,1.12,0.29,2019,2,27,3,369,"[-96.358413, 42.54202]",-96.358413,42.54202
18769033,INV-17819300021,2019-02-27,5003,Famous Liquors,2604 W Locust St,Davenport,52804,POINT (-90.615542 41.538186),82,Scott,...,63.00,16.64,2019,2,27,3,98,"[-90.615542, 41.538186]",-90.615542,41.538186
18769438,INV-17826200010,2019-02-27,5222,Leo1 / Cedar Rapids,"1500, 1st Ave NE",Cedar Rapids,52402,POINT (-91.652082 41.988229),57,Linn,...,9.00,2.37,2019,2,27,3,61,"[-91.652082, 41.988229]",-91.652082,41.988229


In [15]:
data["latitude"]  = data["latitude"].astype("float")
data["longitude"] = data["longitude"].astype("float")
data_filtered = data.loc[(data['longitude'] > 40.396764305572) & (data['longitude'] < 43.580390855608) & 
            (data['latitude'] < -90.087890625) & (data['latitude'] > -96.767578125)]

#-96.767578125 40.396764305572, -96.767578125 43.580390855608, -90.087890625 43.580390855608, -90.087890625 40.396764305572 
#https://geodata.iowa.gov/dataset/boundary-state-iowa
#original = (3448996, 32), after removing = ((3448152,, 32)
data_filtered.shape# 

(3448152, 32)

In [16]:
data_filtered.columns

Index(['Invoice/Item Number', 'Date', 'Store Number', 'Store Name', 'Address',
       'City', 'Zip Code', 'Store Location', 'County Number', 'County',
       'Category', 'Category Name', 'Vendor Number', 'Vendor Name',
       'Item Number', 'Item Description', 'Pack', 'Bottle Volume (ml)',
       'State Bottle Cost', 'State Bottle Retail', 'Bottles Sold',
       'Sale (Dollars)', 'Volume Sold (Liters)', 'Volume Sold (Gallons)',
       'year', 'month', 'day', 'week_of_day', 'encoded_city', 'Store Loc',
       'latitude', 'longitude'],
      dtype='object')

In [17]:
# check whether volume sold = bottlesold * bottle volume 
kk = data_filtered["Bottles Sold"]*data_filtered["Bottle Volume (ml)"]/1000 
mismatch = kk != data_filtered["Volume Sold (Liters)"] #295009
data_filtered = data_filtered.loc[~mismatch,:]

In [18]:
data_filtered.shape #3377142, 32

(3377142, 32)

In [19]:
def drop_3IQR_val(df, col_name):
    q1 = df[col_name].quantile(0.25)
    q3 = df[col_name].quantile(0.75)
    IQR = q3 - q1
    df_new = df.drop(df[(df[col_name] > q3 + 3*IQR) | (df[col_name] < q1 - 3*IQR)].index)
    print("feature = ",col_name, ":",df.shape[0]- df_new.shape[0],"records removed",df_new.shape[0],"records remaning" )
    return (df_new)

targeted_features = ["Volume Sold (Liters)","State Bottle Cost",
                     "State Bottle Retail","Pack"]
for i in targeted_features:
    data_filtered = drop_3IQR_val(data_filtered, i)
#3125315

feature =  Volume Sold (Liters) : 85933 records removed 3291209 records remaning
feature =  State Bottle Cost : 58701 records removed 3232508 records remaning
feature =  State Bottle Retail : 553 records removed 3231955 records remaning
feature =  Pack : 106640 records removed 3125315 records remaning


In [20]:
num_store_per_county = data_filtered[['Store Number','County Number',"Invoice/Item Number"]].groupby(["County Number","Store Number"])
z = list(num_store_per_county.groups.keys())
store_num_counter = Counter(elem[0] for elem in z)
df_store = pd.DataFrame.from_dict(store_num_counter, orient = "index").reset_index()
df_store.columns = ["County Number","store_num"]

data_filtered = pd.merge(data_filtered, df_store, how = "left", on = ['County Number'])
data_filtered.shape #(3125315, 33)

(3125315, 33)

In [21]:
data_filtered = data_filtered[['year', 'month', 'day', 'latitude','longitude',
                            'week_of_day','Pack','City', 'Category','County Number',
                            'County', 'store_num', 'Item Number','Bottle Volume (ml)',
                            'State Bottle Cost', 'State Bottle Retail',
                            'Volume Sold (Liters)','Store Number', 'encoded_city']]
data_filtered.rename(columns = {"County Number":"county_number", 
                             "Volume Sold (Liters)":"volume_sold",
                             "Pack":"pack",
                             "City":"city","Category":"category",
                             "Store Number":"store_number",
                             "State Bottle Retail":"retail_price",
                             "State Bottle Cost":"cost_price",
                             "County Number":"county_number",
                             "County": "county",
                             "Item Number":"item_number",
                             "Bottle Volume (ml)":"bottle_volume"}, inplace = True)

In [22]:
data_filtered["county_number"] = data_filtered["county_number"].astype("category")
data_filtered.dtypes

year                int64
month            category
day              category
latitude          float64
longitude         float64
week_of_day      category
pack                int64
city             category
category         category
county_number    category
county             object
store_num           int64
item_number      category
bottle_volume       int64
cost_price        float64
retail_price      float64
volume_sold       float64
store_number     category
encoded_city     category
dtype: object

In [23]:
data_filtered.year.value_counts()

2019    1750551
2020    1374764
Name: year, dtype: int64

In [24]:
d_2019 = data_filtered.year ==2019
d_2020 = data_filtered.year ==2020

data_2019 = data_filtered.loc[d_2019,:]
data_2020 = data_filtered.loc[~d_2019,:]

data_final_2019 = data_2019.sample(n = 100000, replace = False, random_state = 100)
data_final_2020 = data_2020.sample(n = 100000, replace = False, random_state = 100)

In [25]:
data_final_2019.describe()

,year,latitude,longitude,pack,store_num,bottle_volume,cost_price,retail_price,volume_sold
count,100000.0,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,2019.0,-93.043947,41.966657,11.074600,75.202300,918.862150,1.831995,3.090985,7.015479
std,0.0,1.652388,0.645482,4.649892,74.868225,491.816424,2.320109,2.890465,6.142998
min,2019.0,-96.563230,40.400038,3.000000,1.000000,20.000000,0.010000,0.010000,0.020000
25%,2019.0,-93.825486,41.567246,6.000000,15.000000,750.000000,0.440000,0.740000,1.750000
50%,2019.0,-93.188172,41.828025,12.000000,46.000000,750.000000,0.740000,1.990000,6.000000
75%,2019.0,-91.634871,42.489041,12.000000,120.000000,1000.000000,2.500000,5.500000,10.500000
max,2019.0,-90.182002,43.484108,24.000000,222.000000,2400.000000,9.270000,12.500000,36.000000


In [26]:
data_final_2020.describe()

,year,latitude,longitude,pack,store_num,bottle_volume,cost_price,retail_price,volume_sold
count,100000.0,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,2020.0,-93.055102,41.957378,10.911480,75.830330,900.064300,1.919813,3.160145,6.899978
std,0.0,1.643018,0.647936,4.736899,75.569946,530.422275,2.384035,2.860585,6.133122
min,2020.0,-96.563230,40.400038,1.000000,1.000000,20.000000,0.010000,0.010000,0.020000
25%,2020.0,-93.833463,41.563730,6.000000,15.000000,750.000000,0.470000,0.750000,1.500000
50%,2020.0,-93.202491,41.763876,12.000000,46.000000,750.000000,0.740000,2.380000,6.000000
75%,2020.0,-91.650755,42.484073,12.000000,120.000000,1000.000000,2.670000,5.500000,10.500000
max,2020.0,-90.182002,43.484108,24.000000,222.000000,2400.000000,9.270000,17.740000,36.000000


In [27]:
population_2019 = pd.read_csv("2019_population.csv")
population_2020 = pd.read_csv("2020_population.csv")
population_2019 = population_2019[["population",'county']]
population_2020 = population_2020[["population",'county']]
#population.population = population.population.astype("int")
data_final_2019 = pd.merge(data_final_2019,population_2019,how = "left")
data_final_2020 = pd.merge(data_final_2020,population_2020,how = "left")

In [28]:
data_final_2020.isnull().sum()

year             0
month            0
day              0
latitude         0
longitude        0
week_of_day      0
pack             0
city             0
category         0
county_number    0
county           0
store_num        0
item_number      0
bottle_volume    0
cost_price       0
retail_price     0
volume_sold      0
store_number     0
encoded_city     0
population       0
dtype: int64

In [29]:
data_final_2019.isnull().sum()

year             0
month            0
day              0
latitude         0
longitude        0
week_of_day      0
pack             0
city             0
category         0
county_number    0
county           0
store_num        0
item_number      0
bottle_volume    0
cost_price       0
retail_price     0
volume_sold      0
store_number     0
encoded_city     0
population       0
dtype: int64

In [31]:
data_final_2020

,year,month,day,latitude,longitude,week_of_day,pack,city,category,county_number,county,store_num,item_number,bottle_volume,cost_price,retail_price,volume_sold,store_number,encoded_city,population
0,2020,7,17,-91.136655,40.814666,5,12,Burlington,1062500,29,Des Moines,29,43615,750,0.49,1.24,2.25,2506,50,38774
1,2020,8,13,-93.619787,41.605660,4,12,Des Moines,1032200,77,Polk,222,64573,750,6.49,4.74,1.50,2190,107,499440
2,2020,3,25,-95.916235,41.284951,3,6,Glenwood,1022200,65,Mills,9,88659,750,2.34,8.51,0.75,5259,160,15083
3,2020,2,18,-95.759943,43.403935,2,12,Sibley,1082200,72,Osceola,3,69667,1000,1.00,6.50,4.00,4891,366,6042
4,2020,9,10,-93.396651,43.142775,4,12,Clear Lake,1081400,17,Cerro Gordo,23,82846,750,0.50,0.75,9.00,4969,73,41947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2020,7,17,-91.551598,40.965272,5,12,Mount Pleasant,1022200,44,Henry,8,88294,375,4.50,1.75,2.25,5439,278,20081
99996,2020,1,2,-91.680734,42.034748,4,12,Cedar Rapids,1031100,57,Linn,120,37886,750,0.38,0.07,9.00,3628,61,229173
99997,2020,6,22,-91.591868,41.682337,1,24,Coralville,1062200,52,Johnson,80,43124,375,0.50,0.75,1.50,2670,85,155354
99998,2020,3,10,-93.613739,41.605720,2,3,Des Moines,1022200,77,Polk,222,88298,1750,8.00,7.00,5.25,4829,107,499440


In [32]:
data_final_2019.to_csv("data_2019_100k.csv")
data_final_2020.to_csv("data_2020_100k.csv")